# Zero to Hero

## 1.15 Montecarlo Estimation, Arbol
## En una funcion

El objetivo es implementar con un for loop la estimacion montecarlo, que promedia las ganancias

In [1]:
rm( list=ls())
gc()

library("data.table")   #cargo la libreria  data.table
library("rpart")  #cargo la libreria  rpart
library("rpart.plot")
library("caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("~/lab1/") #Aqui se debe poner la ruta de la PC local

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604245,32.3,1291145,69,1291145,69.0
Vcells,1086153,8.3,8388608,64,1633205,12.5


Loading required package: ggplot2

Loading required package: lattice



La funcion que definimos en el notebook anterior

In [2]:
GananciaArbol  <-  function( semilla, data, x, train=0.70) {
  #establezco la semilla
  set.seed(semilla)
  train_rows <- createDataPartition(dataset$clase_ternaria, p= 0.70,list= FALSE)
  
  modelo <-  rpart::rpart(formula= "clase_ternaria ~ ." ,
                          data= data[ train_rows],  #los datos de training
                          xval= 0,
                          control= x)
    
  #Aplico el modelo a los datos de testing  
  prediccion  <- predict( modelo, data[ -train_rows], type = "prob")
  
  prob_baja2  <- prediccion[, "BAJA+2"]
  ganancia_testing <- data[ -train_rows, sum(  (prob_baja2> 0.025) * ifelse( clase_ternaria=="BAJA+2", 78000, -2000) )]
  
  #normalizo la ganancia
  ganancia_testing_normalizada  <- ganancia_testing/0.3  

  return( ganancia_testing_normalizada )

}

In [3]:
ArbolMontecarlo <- function( semillas, data, x, train=0.70)
{
  vector_ganancias <- c()  #vector donde voy a ir acumulando las ganancias
  for( semilla in ksemillas)
  {
    ganancia  <- GananciaArbol( semilla, dataset, x=x, train=0.70 )
    vector_ganancias  <-  c( vector_ganancias, ganancia)
  }
    
  return( mean( vector_ganancias))
}

In [4]:
dataset <- fread("./datasets/competencia1_2022_v2.csv")   #cargo el dataset #cargo el dataset
dataset <- dataset[ foto_mes==202101 ]  # me quedo solo con el periodo 202101

In [5]:
ksemillas  <- c(999953, 999959, 999961, 999979, 999983) #reemplazar por las propias semillas

In [6]:
#defino el primer set de hiperparametros
param1  <- list("cp"= -0.937,
                "minsplit"=  1143,
                "minbucket"= 1,
                "maxdepth"= 9 )

llamo a la fucion  ArbolMontecarlo con las 5 semillas de ksemillas

In [7]:
ganancia_montecarlo1  <- ArbolMontecarlo( ksemillas, dataset, x= param1, train= 0.70 )

In [8]:
#el SEGUNDO set de hiperparametros
param2  <- list("cp"= -0.819,
                "minsplit"=  830,
                "minbucket"=  1,
                "maxdepth"= 10 )

In [9]:
ganancia_montecarlo2  <- ArbolMontecarlo( ksemillas, dataset, x= param2, train= 0.70 )

In [10]:
ganancia_montecarlo1
ganancia_montecarlo2

[1] 20386667

[1] 20164000

Segun este experimento, el primer juego de hiperparametros es mejor que el primero
<br>
¿Se cumplirá esto subiendo a Kaggle?
<br>
¿Qué sucede si en Kaggle funciona bastante mejor el segundo set de hiperparámetros?